### testing sentinel-1 to detect flooded forests
- sentinel-1 rtc won't load into DEA sandbox environment (seems it can only be accessed within MPC? unconfirmed)
- just plot here the geojson area for specified years to find highest/lowest tides
- using the info to then get sen-1 from MPC to see any discernible backscatter signal of flooded forests

##### 25.1.24 parking this idea, as really need sen-1 data labelled with tide height to be useful #####

In [1]:
import sys

import ipyleaflet
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import geopandas as gpd
import pystac
import pystac_client
import planetary_computer
import requests
# import rich.table

import odc.stac
from datacube.utils.cog import write_cog

from scipy.ndimage import uniform_filter
from scipy.ndimage import variance

from IPython.display import Image

### Tide modelling tools
sys.path.insert(1, "/home/jovyan/Tools/") # needed pip3 install OWSLib
from dea_tools.coastal import model_tides,tidal_tag, pixel_tides, tidal_stats



# GRD perhaps?
# VV seems a winner
# plotting up FES2014 model outside of DEA requires a bit of setup - maybe do within for now either two notebooks or bring this into dea


In [3]:
# Defining a file path
vector_file = '../data/geojson/hawksnest.geojson'
# Define time period of interest
time = "2020-02-07/2020-02-28"


gdf = gpd.read_file(vector_file)

# Visualizing 
gdf.explore()

In [ ]:
# Convert data-cube style queries into something readable by `pystac_client`
bbox = BoundingBox.from_xy(x, y)
time_range = "/".join(time)

# Search for STAC items from "esa-worldcover" product
search = catalog.search(
    collections="esa-worldcover",
    bbox=bbox,
    datetime=time_range,
)

# Check how many items were returned
items = search.item_collection()
print(f"Found {len(items)} STAC items")

In [5]:
gdf_polygon = gdf[gdf.geometry.geom_type == 'Polygon']
# gdf_point = gdf[gdf.geometry.geom_type == 'Point']

polygon_bbox = [gdf_polygon.geometry.bounds.minx.min(), gdf_polygon.geometry.bounds.miny.min(), gdf_polygon.geometry.bounds.maxx.max(), gdf_polygon.geometry.bounds.maxy.max()]
# point_bbox = [gdf_point.geometry.bounds.minx.min(), gdf_point.geometry.bounds.miny.min(), gdf_point.geometry.bounds.maxx.max(), gdf_point.geometry.bounds.maxy.max()]

# search MPC collections
search = catalog.search(
    collections=["sentinel-1-rtc"], bbox=polygon_bbox, datetime=time)
items = search.item_collection()
print(f"Found {len(items)} items")
item = items[0]

Found 1 items


In [ ]:
%%time
# load as odc stac dataset
ds_s1 = odc.stac.load(items,
                        bbox=polygon_bbox,
                        crs="EPSG:3577",
                        resolution=10,
                        groupby='solar_day')

In [ ]:
# Scale to plot data in decibels
ds_s1["vv_dB"] = 10 * np.log10(ds_s1.vv)

# Plot all VV observations for the year
ds_s1.vv_dB.plot(cmap="Greys_r", robust=True, col="time", col_wrap=5)
plt.show()

In [ ]:
%%time
# load as odc stac dataset
ds_s1 = odc.stac.load(items,
                        bbox=polygon_bbox,
                        crs="EPSG:3577",
                        resolution=10,
                        groupby='solar_day')

out_stats = tidal_stats(ds_s1)
